# Exercise 8: Chance-Constrained Economic Dispatch

In [1]:
import gurobipy as gp
from gurobipy import GRB

# Data
demand = 200  # MWh
mu_G2 = 110  # Mean predicted wind generation in MWh
W_G2 = 150   # Total installed wind capacity in MW
scenarios = [-0.09, -0.04, -0.10, 0.08, 0.07, 0.04, 0.06, -0.01, 0.02, 0.07]
probability = 1 / len(scenarios)  # Equal probability for each scenario

# Generator parameters
generators = {
    "G1": {"day_ahead_cost": 75, "up_adjust_cost": 77, "down_adjust_cost": 74,
           "gen_cap": 100, "up_adjust_cap": 10, "down_adjust_cap": 10},
    "G2": {"day_ahead_cost": 6, "up_adjust_cost": 8, "down_adjust_cost": 5,
           "gen_cap": 150, "up_adjust_cap": 150, "down_adjust_cap": 150},
    "G3": {"day_ahead_cost": 80, "up_adjust_cost": 82, "down_adjust_cost": 79,
           "gen_cap": 50, "up_adjust_cap": 50, "down_adjust_cap": 50}
}

# Initialize model
model = gp.Model("economic_dispatch")

# Variables
# Day-ahead generation for each generator
gen_day_ahead = model.addVars(generators.keys(), name="gen_day_ahead")

# Real-time adjustments per scenario (upward and downward)
up_adjust = model.addVars(generators.keys(), range(len(scenarios)), name="up_adjust")
down_adjust = model.addVars(generators.keys(), range(len(scenarios)), name="down_adjust")

# Wind power output per scenario for G2
wind_power = {s: mu_G2 + W_G2 * scenarios[s] for s in range(len(scenarios))}

# Constraints
# 1. Total generation must meet demand in each scenario
for s in range(len(scenarios)):
    model.addConstr(
        gp.quicksum(gen_day_ahead[g] + up_adjust[g, s] - down_adjust[g, s] for g in generators) == demand,
        name=f"demand_scenario_{s}"
    )

# 2. Adjustment limits (upward and downward)
for g in generators:
    for s in range(len(scenarios)):
        model.addConstr(up_adjust[g, s] <= generators[g]["up_adjust_cap"], name=f"up_adjust_cap_{g}_{s}")
        model.addConstr(down_adjust[g, s] <= generators[g]["down_adjust_cap"], name=f"down_adjust_cap_{g}_{s}")

# 3. Generation limits (within generation capacity)
for g in generators:
    for s in range(len(scenarios)):
        model.addConstr(
            gen_day_ahead[g] + up_adjust[g, s] - down_adjust[g, s] <= generators[g]["gen_cap"],
            name=f"gen_cap_{g}_{s}"
        )

# Objective function
# Minimize total expected cost
total_cost = (
    gp.quicksum(generators[g]["day_ahead_cost"] * gen_day_ahead[g] for g in generators) +
    probability * gp.quicksum(
        up_adjust[g, s] * (generators[g]["up_adjust_cost"] - generators[g]["day_ahead_cost"]) +
        down_adjust[g, s] * (generators[g]["day_ahead_cost"] - generators[g]["down_adjust_cost"])
        for g in generators for s in range(len(scenarios))
    )
)
model.setObjective(total_cost, GRB.MINIMIZE)

# Solve
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal Day-ahead Dispatch and Adjustments:")
    for g in generators:
        print(f"Generator {g} Day-ahead Dispatch: {gen_day_ahead[g].X}")
        for s in range(len(scenarios)):
            print(f"  Scenario {s+1}: Up Adjustment = {up_adjust[g, s].X}, Down Adjustment = {down_adjust[g, s].X}")
    print(f"Expected Total Cost: {model.ObjVal}")
else:
    print("No optimal solution found.")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 100 rows, 63 columns and 240 nonzeros
Model fingerprint: 0x68981313
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 60 rows and 0 columns
Presolve time: 0.01s
Presolved: 40 rows, 63 columns, 180 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.500000e+02   0.000000e+00      0s
      10    4.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.000000000e+02
Optimal Day-ahead Dispatch and Adjustme